## Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.


## Descripción de datos

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

- сalls — número de llamadas,

- minutes — duración total de la llamada en minutos,

- messages — número de mensajes de texto,

- mb_used — Tráfico de Internet utilizado en MB,

- is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

## Análisis exploratorio de datos

In [1]:
# Se importan las librerías a utilizar
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# Se lee el dataframe, asi como visualizar sus columnas e información
df = pd.read_csv('users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Segmentación de datos 

In [4]:
df_train, df_temporal = train_test_split(df, test_size=0.4, random_state=54321) #Se separa el df en uno de entrenamiento y uno temporal
df_valid, df_test = train_test_split(df_temporal, test_size=0.5, random_state=54321) #El df temporal se separa en dos nuevamente, el df de validacion y el df de prueba

In [5]:
# Se declaran dos variables para el df de entrenamiento, validación y prueba.
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

## Entrenando diferentes modelos

**Se seleccionaron los modelos de Bosque Aleatorio y Regresión Logística debido a su destacada eficiencia y precisión en comparación con los otros modelos evaluados.**

### Bosque aleatorio

In [6]:
# Se iteran diferentes valores de n_estimators con un bucle y compararemos la calidad de las diferentes versiones del modelo.
best_score = 0
best_est = 0
for est in range(1, 51): # seleccionamos el rango del hiperparámetro
    model = RandomForestClassifier(random_state=54321, n_estimators=est) # configuramos el número de árboles
    model.fit(features_train, target_train) # entrenamos el modelo en el conjunto de entrenamiento
    score = model.score(features_valid, target_valid) # calculamos la puntuación de exactitud en el conjunto de validación
    if score > best_score:
        best_score = score# guardamos la mejor puntuación de exactitud en el conjunto de validación
        best_est = est# se guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 44): 0.7853810264385692


### Regresion logística

In [7]:
# Se prueba ahora el modelo de regresion lógistica para evaluar su exactitud
model = LogisticRegression(random_state=54321, solver='liblinear')# inicializamos el constructor de regresión logística
model.fit(features_train, target_train) # # entrena el modelo en el conjunto de entrenamiento
score_train = model.score(features_train, target_train) # calculamos la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model.score(features_valid, target_valid) # calculamos la puntuación de accuracy en el conjunto de validación

print("Exactitud del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo de regresión logística en el conjunto de validación:", score_valid)

Exactitud del modelo de regresión logística en el conjunto de entrenamiento: 0.7131742738589212
Exactitud del modelo de regresión logística en el conjunto de validación: 0.6780715396578538


**Tras evaluar el desempeño de los distintos modelos, se ha observado que el modelo de Bosque Aleatorio presenta la mayor precisión en los resultados obtenidos. En consecuencia, se ha decidido continuar con su aplicación en el conjunto de prueba para una evaluación más exhaustiva.**

## Modelo con conjunto de prueba

In [8]:
final_model = RandomForestClassifier(random_state=54321, n_estimators=44)
final_model.fit(features_train, target_train)
score_final = final_model.score(test_features, test_target)
predictions = final_model.predict(test_features)

print("La exactitud del modelo en el conjunto de prueba:", score_final)

La exactitud del modelo en el conjunto de prueba: 0.8195956454121306


**Se observa una elevada exactitud con nuestro modelo de bosque aleatorio utilizando el conjunto de prueba, por lo cual, se esta satisfecho con el resultado obtenido**